In [1]:
import numpy as np
import matplotlib.pyplot as plt
import illustris_python as il
from tqdm import tqdm
import os
import pandas as pd
from joblib import Parallel, delayed #This is to parallelize the code
import sys
sys.path.append(os.path.abspath('/bigdata/saleslab/psadh003/tng50-dark-analysis'))
sys.path.append(os.path.abspath('/bigdata/saleslab/psadh003/tng50/dwarf_formation'))
from tng_subhalo_and_halo import TNG_Subhalo
from hydro_path_for_files import * #This is to get the paths for the files
import logging

h = 0.6774
mdm = 4.5e5

In [2]:
ages_df = pd.read_csv('/bigdata/saleslab/psadh003/tng50-dark-analysis/ages_tng.csv', comment = '#') # This is a file that contains redhsifts and ages of universe at different snapshots

all_snaps = np.array(ages_df['snapshot'])
all_redshifts = np.array(ages_df['redshift'])
all_ages = np.array(ages_df['age(Gyr)'])


In [3]:
fofno = 0
this_fof = il.groupcat.loadSingle(basePath, 99, haloID = fofno)
central_id_at99 = this_fof['GroupFirstSub']
this_fof_nsubs = this_fof['GroupNsubs']


In [4]:
central_fields = ['GroupFirstSub', 'SubhaloGrNr', 'SnapNum', 'GroupNsubs', 'SubhaloPos', 'Group_R_Crit200', 'SubhaloVel', 'SubhaloCM', 'Group_M_Crit200'] 
central_tree = il.sublink.loadTree(basePath, 99, central_id_at99, fields = central_fields, onlyMPB = True)
central_snaps = central_tree['SnapNum']
central_redshift = all_redshifts[central_snaps]
central_x =  central_tree['SubhaloPos'][:, 0]/(1 + central_redshift)/h
central_y =  central_tree['SubhaloPos'][:, 1]/(1 + central_redshift)/h
central_z =  central_tree['SubhaloPos'][:, 2]/(1 + central_redshift)/h
central_vx = central_tree['SubhaloVel'][:, 0]
central_vy = central_tree['SubhaloVel'][:, 1]
central_vz = central_tree['SubhaloVel'][:, 2]
Rvir = central_tree['Group_R_Crit200']/(1 + central_redshift)/h #This is the virial radius of the group
ages_rvir = all_ages[central_snaps] #Ages corresponding to the virial radii


In [5]:
def get_grnr(snap):
    '''
    This function returns the group number at a given snapshot 
    snap: input snapshot
    '''
    grnr_arr = central_tree['SubhaloGrNr']
    grnr = grnr_arr[np.where(central_snaps == snap)]
    return grnr


central_grnr = np.zeros(0)

for csnap in central_snaps:
    central_grnr = np.append(central_grnr, get_grnr(csnap))


In [7]:
def get_t1subhalo_data(sfid):  
    '''
    This is for Type-1 in the simulation
    '''
    this_subh = il.groupcat.loadSingle(basePath, 99, subhaloID = sfid)

    if this_subh['SubhaloFlag'] == 0:
        return None

    if_tree = il.sublink.loadTree(basePath, int(99), int(sfid),
            fields = ['SubfindID', 'SnapNum', 'SubhaloMass', 'SubhaloGrNr', 'GroupFirstSub', 'SubhaloVmax', 'SubhaloPos'], 
            onlyMPB = True) #Progenitor tree
    
    if if_tree is None:
        return None # This is to check if the subhalo is not in any tree
    
    cm = np.array(this_subh['SubhaloCM'])/h - (central_x[0], central_y[0], central_z[0])

    subh = TNG_Subhalo(sfid, 99, last_snap = 99)
    pos = subh.get_position_wrt_center(where = 99)

    vel = np.array(this_subh['SubhaloVel'] - (central_vx[0], central_vy[0], central_vz[0]))
    
    pos_ar = pos.reshape(1, -1)
    cm_ar = cm.reshape(1, -1)
    vel_ar = vel.reshape(1, -1)

    # Let us now get the vmax_if by looking at the merger tree
    if_snap = if_tree['SnapNum']
    snap_len = len(if_snap)
    if_grnr = if_tree['SubhaloGrNr']
    # ixs_for_central = np.where(np.isin(central_snaps, if_snap))[0]
    # central_pos = np.column_stack((central_x[ixs_for_central],
    #                                 central_y[ixs_for_central],
    #                                 central_z[ixs_for_central]))
    
    # subh_x = if_tree['SubhaloPos'][:, 0]/h/(1 + all_redshifts[if_snap]) - central_x[ixs_for_central]
    # subh_y = if_tree['SubhaloPos'][:, 1]/h/(1 + all_redshifts[if_snap]) - central_y[ixs_for_central]
    # subh_z = if_tree['SubhaloPos'][:, 2]/h/(1 + all_redshifts[if_snap]) - central_z[ixs_for_central]

    # if_dist = np.sqrt(subh_x**2 + subh_y**2 + subh_z**2)



    i1 = 0
    i2 = 0
    inf1_snap = -1
    inf1_sfid = -1
    matching_snap = -1

    i3 = 0
    crossing_snap = -1
    crossing_sfid = -1


    for ix in range(len(if_snap)):
        '''
        This loop is to go through all the snaps in order to obtain the snap where infall happened
        '''
        snap_ix = if_snap[snap_len - ix - 1] #Go in an ascending order of snapshots for this variable
        
        # print(if_snap, if_grnr, central_grnr)
        
        if (i1 == 0) and (if_tree['SubfindID'][ix] == if_tree['GroupFirstSub'][ix]):
            inf1_snap = if_snap[ix] #This would be the last time when it made transition from central to a satellite
            i1 = 1
            # print(snap, subid, inf1_snap)
        # if (i2 == 0) and (if_grnr[snap_len - ix - 1] == central_grnr[central_snaps == snap_ix]):
        if if_grnr[snap_len - ix - 1].size * central_grnr[central_snaps == snap_ix].size > 0: #What is this for? Assuming this is a check if subhalo existed at this snapshot
            if (i2 == 0) and (if_grnr[snap_len - ix - 1] == central_grnr[central_snaps == snap_ix]):
                matching_snap = snap_ix #This would be the first time when this entered FoF halo 0
                i2 = 1
        # if (i3 == 0) and (if_dist[snap_len - ix - 1] < Rvir[central_snaps == snap_ix]):
        #     crossing_snap = snap_ix
        #     crossing_sfid = if_tree['SubfindID'][snap_len - ix - 1]
        #     i3 = 1
        
    
        if i1*i2 == 1:
            # print(pos_ar[:, 0][0]) 
            # column_names = ['posx_ar', 'posy_ar', 'posz_ar', 'cmx_ar', 'cmy_ar', 'cmz_ar', 'vmax_ar', 'mass_ar', 'len_ar', 'sfid_ar', 
                # 'snap_if_ar', 'sfid_if_ar', 'vmax_if_ar', 'mdm_if_ar', 'inf1_snap_ar', 'inf1_sfid_ar', 'vpeak']
            # In this case matching_snap is the snap where it entered the FoF halo 0
            # Let us calculate the stellar mass at this snapshot
            mstar_infall = subh.get_mstar(where = int(matching_snap), how = 'total')



            # This is to calculate the Vmax at infall with different cases
            if subh.get_mdm(where = int(matching_snap), how = 'total') == 0:
                vmax_at_inf = 0
            else:
                fstar_at_infall = subh.get_mstar(where = int(matching_snap), how = 'vmax')[0]/subh.get_mdm(where = int(matching_snap), how = 'vmax')[0]
                if fstar_at_infall < 0.05:
                    vmax_at_inf = subh.get_vmax(where = int(matching_snap))
                elif fstar_at_infall < 0.1:
                    vmax_at_inf = subh.get_mx_values(where = int(matching_snap), typ = 'dm_dominated')[0]
                elif fstar_at_infall > 0.1:
                    vmax_at_inf = subh.get_mx_values(where = int(matching_snap), typ = 'star_dominated')[0]

            if subh.get_mdm(where = int(99), how = 'total') == 0:
                vmax_at_99 = 0
            else:
                fstar_at_99 = subh.get_mstar(where = int(99), how = 'vmax')[0]/subh.get_mdm(where = int(99), how = 'vmax')[0]
                if fstar_at_99 < 0.05:
                    vmax_at_99 = subh.get_vmax(where = int(99))
                elif fstar_at_99 < 0.1:
                    vmax_at_99 = subh.get_mx_values(where = int(99), typ = 'dm_dominated')[0]
                elif fstar_at_99 > 0.1:
                    vmax_at_99 = subh.get_mx_values(where = int(99), typ = 'star_dominated')[0]
               



            return pos_ar[:, 0][0], pos_ar[:, 1][0], pos_ar[:, 2][0],  vel_ar[:, 0][0], vel_ar[:, 1][0], vel_ar[:, 2][0], sfid, vmax_at_99, matching_snap, if_tree['SubfindID'][if_snap == matching_snap][0], vmax_at_inf
            

            
        
    if i2 ==1 and i1 == 0: # This would be the case wherewe have shifting to the FoF, but it was never a central. It has to be in one of the two cases.
        if subh.get_mdm(where = int(matching_snap), how = 'total') == 0:
                vmax_at_inf = 0
        else:
            fstar_at_infall = subh.get_mstar(where = int(matching_snap), how = 'vmax')[0]/subh.get_mdm(where = int(matching_snap), how = 'vmax')[0]
            if fstar_at_infall < 0.05:
                vmax_at_inf = subh.get_vmax(where = int(matching_snap))
            elif fstar_at_infall < 0.1:
                vmax_at_inf = subh.get_mx_values(where = int(matching_snap), typ = 'dm_dominated')[0]
            elif fstar_at_infall > 0.1:
                vmax_at_inf = subh.get_mx_values(where = int(matching_snap), typ = 'star_dominated')[0]

        if subh.get_mdm(where = int(99), how = 'total') == 0:
            vmax_at_99 = 0
        else:
            fstar_at_99 = subh.get_mstar(where = int(99), how = 'vmax')[0]/subh.get_mdm(where = int(99), how = 'vmax')[0]
            if fstar_at_99 < 0.05:
                vmax_at_99 = subh.get_vmax(where = int(99))
            elif fstar_at_99 < 0.1:
                vmax_at_99 = subh.get_mx_values(where = int(99), typ = 'dm_dominated')[0]
            elif fstar_at_99 > 0.1:
                vmax_at_99 = subh.get_mx_values(where = int(99), typ = 'star_dominated')[0]
            



        return pos_ar[:, 0][0], pos_ar[:, 1][0], pos_ar[:, 2][0],  vel_ar[:, 0][0], vel_ar[:, 1][0], vel_ar[:, 2][0], sfid, vmax_at_99, matching_snap, if_tree['SubfindID'][if_snap == matching_snap][0], vmax_at_inf

In [8]:
for sfid in tqdm(range(571, central_id_at99 + this_fof_nsubs)):
    get_t1subhalo_data(sfid)

# print(get_t1subhalo_data(central_id_at99+1))

  0%|          | 227/63293 [01:22<6:18:41,  2.78it/s]/bigdata/saleslab/psadh003/tng50/dwarf_formation/tng_subhalo_and_halo.py:195: RuntimeWarning: divide by zero encountered in log10
  result = np.array([np.log10(nfw_mass(r1, lrhos, lrs)) - np.log10(float(m1)), np.log10(nfw_mass(r2, lrhos, lrs)) - np.log10(float(m2))]).ravel()
/bigdata/saleslab/psadh003/tng50/dwarf_formation/tng_subhalo_and_halo.py:202: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  lrhos, lrs = fsolve(simul_func, input_values)
  7%|▋         | 4481/63293 [27:15<4:46:08,  3.43it/s]

Warning, empty return. Subhalo [5051] at snapNum [99] not in tree.


 16%|█▌        | 10183/63293 [1:01:32<4:04:34,  3.62it/s] 

Warning, empty return. Subhalo [10753] at snapNum [99] not in tree.


 22%|██▏       | 13788/63293 [1:23:19<4:59:08,  2.76it/s] 


KeyboardInterrupt: 